In [1]:
library(R.matlab)
library(ggplot2)
library(pracma)

R.matlab v3.6.1 (2016-10-19) successfully loaded. See ?R.matlab for help.

Attaching package: ‘R.matlab’

The following objects are masked from ‘package:base’:

    getOption, isOpen



In [28]:
# R code of package 'mnormt' 
# Author: Adelchi Azzalini (University of Padua, Italy) 

dmnorm <- function(x, mean=rep(0,d), varcov, log=FALSE)
{
  d <- if(is.matrix(varcov)) ncol(varcov) else 1
  if(d==1) return(dnorm(x, mean, sqrt(varcov), log=log))
  x <- if (is.vector(x)) t(matrix(x)) else data.matrix(x)
  if(ncol(x) != d) stop("mismatch of dimensions of 'x' and 'varcov'")
  if(is.matrix(mean)) { if ((nrow(x) != nrow(mean)) || (ncol(mean) != d))
      stop("mismatch of dimensions of 'x' and 'mean'") }
  if(is.vector(mean)) mean <- outer(rep(1, nrow(x)), as.vector(matrix(mean,d)))
  X  <- t(x - mean)
  conc <- pd.solve(varcov, log.det=TRUE)
  Q <- colSums((conc %*% X)* X)
  log.det <- attr(conc, "log.det")
  logPDF <- as.vector(Q + d*logb(2*pi) + log.det)/(-2)
  if(log) logPDF else exp(logPDF)
}

rmnorm <- function(n=1, mean=rep(0,d), varcov, sqrt=NULL)
 {
  sqrt.varcov <- if(is.null(sqrt)) chol(varcov) else sqrt
  d <- if(is.matrix(sqrt.varcov)) ncol(sqrt.varcov) else 1
  mean <- outer(rep(1,n), as.vector(matrix(mean,d)))
  drop(mean + t(matrix(rnorm(n*d), d, n)) %*% sqrt.varcov)
 }


pmnorm <- function(x, mean=rep(0, d), varcov, ...) {
  d <- NCOL(varcov)
  x <- if (is.vector(x)) matrix(x, 1, d) else data.matrix(x)
  n <- nrow(x)
  if(is.vector(mean)) mean <- outer(rep(1, n), as.vector(matrix(mean,d)))
  if(d == 1) p <- as.vector(pnorm(x, mean, sqrt(varcov))) else {
    pv <- numeric(n)
    for (j in 1:n) p <- pv[j] <- if(d == 2)
           biv.nt.prob(0, lower=rep(-Inf, 2), upper=x[j,], mean[j,], varcov)   
      else sadmvn(lower=rep(-Inf, d), upper=x[j,], mean[j,], varcov, ...) 
    if(n > 1) p <- pv 
    }
  return(p)  
  }

sadmvn <- function(lower, upper, mean, varcov,  
                   maxpts=2000*d, abseps=1e-6, releps=0)
{
  if(any(lower > upper)) stop("lower>upper integration limits")
  if(any(lower == upper)) return(0)
  d <- as.integer(if(is.matrix(varcov)) ncol(varcov) else 1)
  varcov <- matrix(varcov, d, d)
  sd  <- sqrt(diag(varcov))
  rho <- cov2cor(varcov)
  lower <- as.double((lower-mean)/sd)
  upper <- as.double((upper-mean)/sd)
  if(d == 1) return(pnorm(upper) - pnorm(lower))
  infin <- rep(2,d)
  infin <- replace(infin, (upper == Inf) & (lower > -Inf), 1)
  infin <- replace(infin, (upper < Inf) & (lower == -Inf), 0)
  infin <- replace(infin, (upper == Inf) & (lower == -Inf), -1)
  infin <- as.integer(infin)
  if(any(infin == -1)) {
    if(all(infin == -1)) return(1)
    k <- which(infin != -1)
    d <- length(k)
    lower <- lower[k]
    upper <- upper[k]
    if(d == 1) return(pnorm(upper) - pnorm(lower))
    rho <- rho[k, k]
    infin <- infin[k]
    if(d == 2) return(biv.nt.prob(0, lower, upper, rep(0,2), rho))
    }
  lower <- replace(lower, lower == -Inf, 0)
  upper <- replace(upper, upper == Inf, 0)
  correl <- as.double(rho[upper.tri(rho, diag=FALSE)])
  maxpts <- as.integer(maxpts)
  abseps <- as.double(abseps)
  releps <- as.double(releps)
  error  <- as.double(0)
  value  <- as.double(0)
  inform <- as.integer(0)
  result <- .Fortran("sadmvn", d, lower, upper, infin, correl, maxpts,
               abseps, releps, error, value, inform, PACKAGE="mnormt")
  prob <- result[[10]]
  attr(prob,"error")  <- result[[9]]
  attr(prob,"status") <- switch(1 + result[[11]], 
                "normal completion", "accuracy non achieved", "oversize")
  return(prob)
}

#----

dmt <- function (x, mean=rep(0,d), S, df = Inf, log = FALSE) 
{
  if (df == Inf)  return(dmnorm(x, mean, S, log = log))
  d <- if(is.matrix(S)) ncol(S) else 1
  if (d==1) {
    y <- dt((x-mean)/sqrt(S), df=df, log=log)
    if(log) y <- (y - 0.5*logb(S)) else y <- y/sqrt(S)
    return(y)
    }
  x <- if (is.vector(x)) t(matrix(x)) else data.matrix(x)
  if (ncol(x) != d) stop("mismatch of dimensions of 'x' and 'varcov'")
  if (is.matrix(mean)) {if ((nrow(x) != nrow(mean)) || (ncol(mean) != d))
      stop("mismatch of dimensions of 'x' and 'mean'") }
  if(is.vector(mean)) mean <- outer(rep(1, nrow(x)), as.vector(matrix(mean,d)))
  X  <- t(x - mean)
  S.inv <- pd.solve(S, log.det=TRUE)
  Q <- colSums((S.inv %*% X) * X)
  logDet <- attr(S.inv, "log.det")
  logPDF <- (lgamma((df + d)/2) - 0.5 * (d * logb(pi * df) + logDet)
             - lgamma(df/2) - 0.5 * (df + d) * logb(1 + Q/df))
  if(log) logPDF else exp(logPDF)
}


rmt <- function(n=1, mean=rep(0,d), S, df=Inf, sqrt=NULL)
{ 
  sqrt.S <- if(is.null(sqrt)) chol(S) else sqrt
  d <- if(is.matrix(sqrt.S)) ncol(sqrt.S) else 1 
  x <- if(df==Inf) 1 else rchisq(n, df)/df
  z <- rmnorm(n, rep(0, d), sqrt=sqrt.S)
  mean <- outer(rep(1, n), as.vector(matrix(mean,d)))
  drop(mean + z/sqrt(x))
}
 


pmt <- function(x, mean=rep(0, d), S, df=Inf, ...){
  d <- NCOL(S)
  x <- if(is.vector(x)) matrix(x, 1, d) else data.matrix(x)
  n <- nrow(x)
  if(is.vector(mean)) mean <- outer(rep(1, n), as.vector(matrix(mean,d)))
  if(d == 1) p <- as.vector(pt((x-mean)/sqrt(S), df=df)) else {
    pv <- numeric(n)
    for (j in 1:n) p <- pv[j] <- if(d == 2)
           biv.nt.prob(df, lower=rep(-Inf, 2), upper=x[j,], mean[j,], S)   
      else sadmvt(df, lower=rep(-Inf, d), upper=x[j,], mean[j,], S, ...)
     if(n > 1) p <- pv    
     }
  return(p)  
  }
   


sadmvt <- function(df, lower, upper, mean, S, 
                   maxpts=2000*d, abseps=1e-6, releps=0)
{
  if(df == Inf) return(sadmvn(lower, upper, mean, S, maxpts, abseps, releps))
  if(any(lower > upper)) stop("lower>upper integration limits")
  if(any(lower == upper)) return(0)
  if(round(df) != df) warning("non integer df is rounded to integer") 
  df <- as.integer(round(df))
  d  <- as.integer(if(is.matrix(S)) ncol(S) else 1)
  S  <- matrix(S, d, d)
  sd  <- sqrt(diag(S))
  rho <- cov2cor(S)
  lower <- as.double((lower-mean)/sd)
  upper <- as.double((upper-mean)/sd)
  if(d == 1) return(pt(upper, df) - pt(lower, df))
  infin <- rep(2,d)
  infin <- replace(infin, (upper == Inf) & (lower > -Inf), 1)
  infin <- replace(infin, (upper < Inf) & (lower == -Inf), 0)
  infin <- replace(infin, (upper == Inf) & (lower == -Inf), -1)
  infin <- as.integer(infin)
  if(any(infin == -1)) {
    if(all(infin == -1)) return(1)
    k <- which(infin != -1)
    d <- length(k)
    lower <- lower[k]
    upper <- upper[k]
    if(d == 1) return(pt(upper, df=df) - pt(lower, df=df))
    rho <- rho[k, k]
    infin <- infin[k]
    if(d == 2) return(biv.nt.prob(df, lower, upper, rep(0,2), rho))
    }
  lower <- replace(lower, lower == -Inf, 0)
  upper <- replace(upper, upper == Inf, 0)
  correl <- rho[upper.tri(rho, diag=FALSE)]
  maxpts <- as.integer(maxpts)
  abseps <- as.double(abseps)
  releps <- as.double(releps)
  error  <- as.double(0)
  value  <- as.double(0)
  inform <- as.integer(0)
  result <- .Fortran("sadmvt", d, df, lower, upper, infin, correl, maxpts,
                   abseps, releps, error, value, inform, PACKAGE="mnormt")
  prob <- result[[11]]
  attr(prob,"error")  <- result[[10]]
  attr(prob,"status") <- switch(1+result[[12]], 
                "normal completion", "accuracy non achieved", "oversize")
  return(prob)
}


biv.nt.prob <- function(df, lower, upper, mean, S){
  if(any(dim(S) != c(2,2))) stop("dimensions mismatch")
  if(length(mean) != 2) stop("dimensions mismatch") 
  if(round(df) != df) warning("non integer df is rounded to integer") 
  nu <- if(df<Inf) as.integer(round(df)) else 0
  if(df==Inf) nu <- 0
  sd <- sqrt(diag(S))
  rho <- cov2cor(S)[1,2]
  lower <- as.double((lower-mean)/sd)
  upper <- as.double((upper-mean)/sd)
  if(any(lower > upper)) stop("lower>upper integration limits")
  if(any(lower == upper)) return(0)
  infin <- c(2,2)
  infin <- replace(infin, (upper == Inf) & (lower > -Inf), 1)
  infin <- replace(infin, (upper < Inf) & (lower == -Inf), 0)
  infin <- replace(infin, (upper == Inf) & (lower == -Inf), -1)
  infin <- as.integer(infin)
  if(any(infin == -1)) {
    if(all(infin == -1)) return(1)
    k <- which(infin != -1)
    return(pt(upper[k], df=df) - pt(lower[k], df=df))
    }
  lower <- replace(lower, lower == -Inf, 0)
  upper <- replace(upper, upper == Inf, 0)
  rho   <- as.double(rho)
  prob  <- as.double(0)
  a <- .Fortran("smvbvt", prob, nu, lower, upper, infin, rho, PACKAGE="mnormt")
  return(a[[1]])
  } 
 
pd.solve <- function(x, silent=FALSE, log.det=FALSE)
{
  if(is.null(x)) return(NULL)
  if(any(is.na(x)))
    {if(silent) return (NULL) else stop("NA's in x") } 
  if(length(x) == 1) x <- as.matrix(x)
  if(!is.matrix(x)) 
    {if(silent) return(NULL) else stop("x is not a matrix")}
  if(max(abs(x - t(x))) > .Machine$double.eps) 
    {if(silent) return (NULL) else stop("x appears to be not symmetric") } 
  x <- (x + t(x))/2
  u <- try(chol(x, pivot = FALSE), silent = silent)
  if(class(u) == "try-error") {
     if(silent) return(NULL) else
       stop("x appears to be not positive definite") }
  inv <- chol2inv(u)
  if(log.det) attr(inv, "log.det") <- 2 * sum(log(diag(u)))
  dimnames(inv) <- rev(dimnames(x))
  return(inv)
}

.onLoad <- function(library, pkg)
{ 
   library.dynam("mnormt", pkg, library)
   invisible()
}

In [49]:
init_f <- function(Z, k){
  n = ncol(Z)
  f_init = matrix(0, n + k, 1)
  Z_t = t(Z)

  mzt = rowMeans(Z_t)
  mzt = as.matrix(mzt)

  Z_new = matrix(0, dim(Z_t)[1], dim(Z_t)[2])

  for(i in 1:dim(Z_t)[2]) {
    Z_new[, i] = Z_t[, i] - mzt
  }

  #z_new = sweep(z, 2, rowMeans(z), "-")
  Z_svd = svd(Z_new)
  f_init[1:n, ] = Z_new %*% as.matrix(Z_svd$v[, 1])

  f_init = f_init/sd(f_init)
  return (f_init)
}

In [3]:
run_train_step <- function(z, k, f, alpha, beta) {
  m = nrow(z);
  T = ncol(z);
  ab = alpha_beta(z, f, k);
  alpha_new = ab$alpha;
  beta_new = ab$beta;
  f_star = f_alpha_beta(z, k, alpha_new, beta_new);
  f_centered = f_star -  mean(f_star);
  f_new = sqrt(T + k) * f_centered / max(svd(f_centered)$d); ########
  return(list("f_new"=f_new, "alpha_new" = alpha_new, "beta_new" = beta_new))
}

In [4]:
evaluate_op <- function(z, k, f, alpha, beta){
  loss = mean_squared_error(z, k, f, alpha, beta)
  return (loss)
}

In [5]:
mean_squared_error <- function(z, k, f, alpha, beta){
  m = nrow(z);
  T = ncol(z);
  sum_squared_error = 0;
  for (j in 1:m){
    for (t in 1:T){
      z_jt_predict = alpha[j];
      for (i in 0:k){
        z_jt_predict = z_jt_predict + beta[j, i + 1] * f[t + i];
      }
      sum_squared_error = sum_squared_error + (z[j, t] - z_jt_predict)^2;
    }
  }
  err = sum_squared_error / (T * m);
  return(err);
}

In [6]:
C_alpha <- function(z_j, alpha_j, k){
  T = ncol(z_j);
  C = matrix(0, T + k, k + 1);
  for (t in 1:(T + k)){
    for (q in 1:(k + 1)){
      if ((q >= max(1, t - T + 1)) && (q <= min(k + 1, t))){
        C[t, q] = z[t - q + 1] - alpha_j
      }
    }
  }
  return (C)
}

In [7]:
D_beta <- function(beta_j, T, k) {
  # Construct matrix D w.r.t beta_j, T, and k
  # beta_j is a row vector of the Beta matrix
  D = matrix(0, T+k, T+k)
  for (t in 1:T+k) {
    for (q in 1:T+k) {
        if ((q >= max(t - k, 1)) && (q <= min(t + k, T + k))) {
            for (v in max(t - k, 1):min(t, T)) {
                D[t,q] = D[t,q] + beta_j[q-v+1] * beta[t-v+1]
            }
        }
    }
  }
   print(D)
  return (D)
}

In [8]:
f_alpha_beta <- function(Z, alpha, beta, k){
  # Get optimal f wrt Z, beta, alpha, and k
  print(dim(beta))
  m = nrow(Z)
  T = ncol(Z)
  D = matrix(0, T+k, T+k)
  f = matrix(0, T+k, 1)
  for (j in 1:m) {
    D = D + D_beta(beta[j,], T, k)
    f = f + C_alpha(Z[j,], alpha[j]) %*% beta[j,]
  }
  f <- ifelse(rcond(D)>1e-10, solve(D,f), pinv(D))
  return (f)
}


In [9]:
F_f <- function(Z, f, k){
  # Gets F matrix s.t. F(f) is T x (k+2) w/ t-th row (f_t, f_t+1,...,f_t+k, 1)
  m = nrow(Z);
  T = ncol(Z);
  F = matrix(0,nrow=T,ncol=k+2)
  for (t in 1:nrow(F)){
    F[t, ] = cbind(Conj(t(f[t:(t + k)])), 1);
  }
  return(F);
}


In [81]:
alpha_beta <- function(Z, f, k){
  # Find the optimal beta
  # input: Z data matrix, f-principal components, k-leads
  # output: beta matrix of loadings, alpha-intercepts
  #F=F_f(Z, f, k)
  #FtF_inv = solve(Conj(t(F)) %*% F);
  #FtF_inv_Ft = FtF_inv %*% Conj(t(F));
  #tmp = Z %*% Conj(t(FtF_inv_Ft));
  #alpha = Conj(t(tmp[, k + 2]));
  #beta = tmp[, 1:(k + 1)];
  #res = Z - beta %*% F
    m = nrow(Z)
    T = ncol(Z)
    F = F_f(Z, f, k)
    FF = t(F) %*% F
    FFinv = pinv(FF)
    FFinv_Ft = FFinv %*% t(F)
    beta = Z %*% t(FFinv_Ft)
    #beta = Z %*% t(F) %*% t(FFinv)
    return(beta)

    
  #return(list("alpha"=alpha, "beta"=beta))
}

In [11]:
run_gdpc <- function(Z, k, f_ini=NULL, tol=1e-04, maxiter=100) {
  # Each ROW is a DIFFERENT time series
  # run the DPC model
  m = nrow(z);
  T = ncol(z);


  f = ifelse(is.null(f_ini), init_f(Z, k), f_ini)
  alpha = NaN
  beta = NaN
  loss_values = matrix(0, nrow=maxiter, ncol=1)

  for (train_iter in 1:train_iterations) {
    a = run_train_step(z, k, f, alpha, beta);
    f = a$f_new;
    alpha = a$alpha_new;
    beta = a$beta_new
    loss_values[train_iter] = evaluate_op(z, k, f, alpha, beta);
  }

  ## plot result
  #plot(loss_values, lwd = 1.5, main="train loss (reconstruction MSE)", xlab = "iteration");
  #print(loss_values)

  #plot(f, lwd = 1.5, main = "Factor", xlab = "time" );
  #lines(f, lty = 1)
}

In [41]:
data = readMat('F141020-lfp-5min-1kHz.mat')

In [42]:
pre_pmcao = data$pre.pmcao
pre_epoch1 = pre_pmcao[1:1000, ]

In [43]:
dim(pre_epoch1)

[1] 1000   32

In [50]:
k = 2
f = init_f(t(pre_epoch1), k)

In [51]:
dim(f)

[1] 1002    1

In [52]:
maxiter = 100
alpha = NaN
beta = NaN
loss_values = matrix(0, maxiter, 1)

In [82]:
s = alpha_beta(t(pre_epoch1), f, k)

In [83]:
dim(s)

[1] 32  4

In [59]:
# for (niter in 1:maxiter) {
#     a = run_train_step(Z_t, k, f, alpha, beta);
#     f = a$f_new;
#     alpha = a$alpha_new;
#     beta = a$beta_new
#     loss_values[train_iter] = evaluate_op(Z_t, k, f, alpha, beta);
# }

NULL
